#Featurizing text data with tfidf

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics.classification import accuracy_score, log_loss

In [0]:
dfnlp = pd.read_csv("/content/drive/My Drive/Quora/nlp_features_train.csv",encoding='latin-1')
dfppro = pd.read_csv("/content/drive/My Drive/Quora/df_fe_without_preprocessing_train.csv",encoding='latin-1')
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)

In [3]:
df= pd.read_csv("/content/drive/My Drive/Quora/train.csv")

df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [0]:
df3 = df.drop(['qid1','qid2','is_duplicate'],axis=1)
df1  = df1.merge(df2, on='id',how='left')
df1 = df1.merge(df3, on='id',how='left')

In [5]:
y = df1.is_duplicate[:10000]
df1 = df1.drop(['is_duplicate'],axis=1)[:10000]
df1.head(2)

,id,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2,question1,question2
0,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759,1,1,66,57,14,12,10.0,23.0,0.434783,2,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,1,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154,4,1,51,88,8,13,4.0,20.0,0.200000,5,3,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...


In [0]:
#splitting into train and test, We are only sampling 10000 data points
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( df1, y, test_size=0.3, random_state=42)

In [7]:
print("shape of training data ",X_train.shape)
print("shape of testing data ",X_test.shape)

shape of training data  (7000, 29)
shape of testing data  (3000, 29)


In [8]:
#training the tf idf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

questions = list(X_train['question1']) + list(X_train['question2'])
tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

<14000x14172 sparse matrix of type '<class 'numpy.float64'>'
	with 140673 stored elements in Compressed Sparse Row format>

In [9]:
#transforming training data
tfidf_q1= tfidf.transform(X_train['question1']).toarray()
tfidf_q2= tfidf.transform(X_train['question2']).toarray()
X_train= X_train.drop(['question1','question2'],axis=1)
X_train=np.hstack((X_train.values,tfidf_q1,tfidf_q2))
print("shape of our training data ", X_train.shape)

shape of our training data  (7000, 28371)


In [10]:

#transforming testing data
tfidf_q1= tfidf.transform(X_test['question1']).toarray()
tfidf_q2= tfidf.transform(X_test['question2']).toarray()
X_test= X_test.drop(['question1','question2'],axis=1)
X_test=np.hstack((X_test.values,tfidf_q1,tfidf_q2))
print("shape of our testing data ", X_testing.shape)


shape of our testing data  28371


In [0]:
# This function plots the confusion matrices given y_i, y_i_hat.
def plot_confusion_matrix(test_y, predict_y):
    C = confusion_matrix(test_y, predict_y)
    # C = 9,9 matrix, each cell (i,j) represents number of points of class i are predicted class j
    
    A =(((C.T)/(C.sum(axis=1))).T)
    #divid each element of the confusion matrix with the sum of elements in that column
    
    # C = [[1, 2],
    #     [3, 4]]
    # C.T = [[1, 3],
    #        [2, 4]]
    # C.sum(axis = 1)  axis=0 corresonds to columns and axis=1 corresponds to rows in two diamensional array
    # C.sum(axix =1) = [[3, 7]]
    # ((C.T)/(C.sum(axis=1))) = [[1/3, 3/7]
    #                           [2/3, 4/7]]

    # ((C.T)/(C.sum(axis=1))).T = [[1/3, 2/3]
    #                           [3/7, 4/7]]
    # sum of row elements = 1
    
    B =(C/C.sum(axis=0))
    #divid each element of the confusion matrix with the sum of elements in that row
    # C = [[1, 2],
    #     [3, 4]]
    # C.sum(axis = 0)  axis=0 corresonds to columns and axis=1 corresponds to rows in two diamensional array
    # C.sum(axix =0) = [[4, 6]]
    # (C/C.sum(axis=0)) = [[1/4, 2/6],
    #                      [3/4, 4/6]] 
    plt.figure(figsize=(20,4))
    
    labels = [0,1]
    # representing A in heatmap format
    cmap=sns.light_palette("blue")
    plt.subplot(1, 3, 1)
    sns.heatmap(C, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Confusion matrix")
    
    plt.subplot(1, 3, 2)
    sns.heatmap(B, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Precision matrix")
    
    plt.subplot(1, 3, 3)
    # representing B in heatmap format
    sns.heatmap(A, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Recall matrix")
    
    plt.show()

##

In [19]:
clf = SGDClassifier(alpha=i, penalty='l2', loss='log', random_state=42)
clf.fit(X_train, y_train)
sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
sig_clf.fit(X_train, y_train)
predict_y = sig_clf.predict_proba(X_test)
log_error_array.append(log_loss(y_test, predict_y, labels=clf.classes_, eps=1e-15))
print('For values of alpha = ', i, "The log loss is:",log_loss(y_test, predict_y, labels=clf.classes_, eps=1e-15))




predict_y = sig_clf.predict_proba(X_train)
print('For values of best alpha = ', alpha[best_alpha], "The train log loss is:",log_loss(y_train, predict_y, labels=clf.classes_, eps=1e-15))
predict_y = sig_clf.predict_proba(X_test)
print('For values of best alpha = ', alpha[best_alpha], "The test log loss is:",log_loss(y_test, predict_y, labels=clf.classes_, eps=1e-15))
predicted_y =np.argmax(predict_y,axis=1)
print("Total number of data points :", len(predicted_y))
plot_confusion_matrix(y_test, predicted_y)

For values of alpha =  1e-05 The log loss is: 0.6584430176553845


NameError: ignored

,id,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2,question1,question2
9069,9069,0.333322,0.249994,0.599988,0.599988,0.499994,0.444440,1.0,1.0,1.0,8.5,72,62,54,53,0.250000,1,1,55,47,8,8,3.0,16.0,0.187500,2,0,What are unknown and/or amazing facts about Sa...,What is your review about Slack and Salesforce?
2603,2603,0.499988,0.333328,0.999980,0.833319,0.699993,0.538457,0.0,1.0,3.0,11.5,79,71,71,71,0.461538,1,1,66,51,13,10,7.0,21.0,0.333333,2,0,What is the formula for moment of inertia of a...,What is the moment of inertia of a hexagonal p...
7738,7738,0.999980,0.833319,0.999989,0.999989,0.999993,0.933327,1.0,1.0,1.0,14.5,100,97,98,95,0.617284,1,1,78,82,13,13,12.0,26.0,0.461538,2,0,What is the best relationship (romantic) you'v...,What is the best relationship (non-romantic) y...
1579,1579,0.999975,0.799984,0.999980,0.999980,0.999989,0.899991,1.0,1.0,1.0,9.5,100,93,93,86,0.659091,1,1,43,49,9,10,9.0,19.0,0.473684,2,0,Why did Sirius have to travel north in GOF?,Why did Sirius Black have to travel north in GOF?
5058,5058,0.000000,0.000000,0.249994,0.199996,0.111110,0.099999,0.0,1.0,1.0,9.5,48,48,48,48,0.078431,1,1,50,52,10,9,1.0,19.0,0.052632,2,0,How do I achieve good postion being as ETL tes...,How competitive is the hiring process at John ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,5734,0.499992,0.499992,0.749981,0.749981,0.599994,0.599994,1.0,1.0,0.0,10.0,75,72,70,64,0.222222,8,8,53,58,10,10,6.0,20.0,0.300000,16,0,Where can I buy special flavor cupcake at Gold...,Where can I found different cupcake flavors in...
5191,5191,0.333322,0.249994,0.000000,0.000000,0.142855,0.142855,0.0,0.0,0.0,7.0,49,47,47,49,0.210526,1,1,42,37,7,7,1.0,14.0,0.071429,2,0,Who were the most notorious usenet trolls?,Could Quora have been a Usenet group?
5390,5390,0.249994,0.142855,0.399992,0.333328,0.333330,0.230767,0.0,0.0,4.0,11.0,52,43,45,50,0.159091,1,1,66,43,12,7,2.0,19.0,0.105263,2,0,How can I restore my lost and unsaved drawing ...,What should I draw/sketch/paint on my door?
860,860,0.833319,0.833319,0.666644,0.285710,0.777769,0.538457,0.0,0.0,4.0,11.0,86,78,53,63,0.461538,1,1,51,38,13,9,5.0,22.0,0.227273,2,0,How do you get a micro SIM card out of an iPho...,Does an iPhone 4 use a micro SIM card?
